## Import modules


In [ ]:
from spezi_data_pipeline.data_access.firebase_fhir_data_access import FirebaseFHIRAccess, FHIRResourceType
from spezi_data_pipeline.data_flattening.fhir_resources_flattener import extract_questionnaire_mappings, flatten_fhir_resources, FHIRDataFrame, QuestionnaireResponseFlattener
from spezi_data_pipeline.data_processing.data_processor import FHIRDataProcessor
from spezi_data_pipeline.data_processing.questionnaire_processor import calculate_risk_score
from spezi_data_pipeline.data_exploration.data_explorer import DataExplorer, visualizer_factory, explore_total_records_number
from spezi_data_pipeline.data_export.data_exporter import DataExporter

## Define credential files

In [ ]:
# Define your Firebase project ID
project_id = "spezi-data-pipeline"  # Replace with your Firebase project ID

# Define the service account key file
service_account_key_file = "path_to_service_account_key_file.json"  # Replace with your service account key file

# Define the collection name where your FHIR observations are stored and the input code if filtering is needed
collection_name = "users"
subcollection_name = "QuestionnaireResponse"

# Define the survey path for creating questionnaire mappings
questionnaire_resource_path = "path_to_the_questionnaire_resource_file.json"

## Initialize the FirebaseFHIRAccess class using your Firebase credentials

In [ ]:
# Initialize the FirebaseFHIRAccess class using your Firebase credentials
firebase_access = FirebaseFHIRAccess(project_id, service_account_key_file)
firebase_access.connect()

fhir_questionnaires = firebase_access.fetch_data(collection_name, subcollection_name)

flattened_fhir_dataframe = flatten_fhir_resources(fhir_questionnaires, questionnaire_path)

In [ ]:
flattened_fhir_dataframe.df.head(15)

## Explore Data

## Export data

In [ ]:
exporter = DataExporter(flattened_fhir_dataframe)
exporter.export_to_csv("survey_data.csv") 

# PHQ-9 example

In [ ]:
import json
import numpy as np
from fhir.resources.bundle import Bundle
from fhir.resources.questionnaireresponse import QuestionnaireResponse

def load_bundle_from_json(file_path: str) -> Bundle:
    with open(file_path, 'r', encoding='utf-8') as f:
        json_content = json.load(f)
    try:
        bundle = Bundle.parse_obj(json_content)
    except FHIRValidationError as e:
        print(f"Failed to parse Bundle: {e}")
        bundle = None
    return bundle

def extract_questionnaire_responses(bundle: Bundle) -> list:
    questionnaire_responses = []
    for entry in bundle.entry:
        if entry.resource.resource_type == "QuestionnaireResponse":
            try:
                qr = QuestionnaireResponse.parse_obj(entry.resource.dict())
                questionnaire_responses.append(qr)
            except FHIRValidationError as e:
                print(f"Failed to parse QuestionnaireResponse: {e}")
    return questionnaire_responses

file_path = 'sample_data/phq9responses.json'

bundle = load_bundle_from_json(file_path)

if bundle:
    questionnaire_responses = extract_questionnaire_responses(bundle)
    survey_path = 'Resources/PHQ-9.json'
    flattener = QuestionnaireResponseFlattener()
    
    flattened_fhir_dataframe = flattener.flatten(questionnaire_responses, survey_path)

flattened_fhir_dataframe.df.tail()

## Calculate risk score

In [ ]:
questionnaire_title = "PHQ-9"
flattened_fhir_dataframe_with_score = calculate_risk_score(flattened_fhir_dataframe, questionnaire_title)

random_user_ids = np.random.randint(100000, 999999, size=len(flattened_fhir_dataframe_with_score.df))
flattened_fhir_dataframe_with_score.df['UserId'] = random_user_ids
flattened_fhir_dataframe_with_score.df['ResourceId'] = random_user_ids
flattened_fhir_dataframe_with_score.df

In [ ]:
explorer = visualizer_factory(flattened_fhir_dataframe_with_score, questionnaire_title="PHQ-9")
# explorer.set_user_ids(["User1", "User2"])
# explorer.set_date_range("2023-01-13", "2023-02-16")
fig = explorer.create_score_plot(flattened_fhir_dataframe_with_score)